# SVM Classification using Only GIST Features

## Initial Setup

In [1]:
from __future__ import division
from PIL import Image
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import gist
import imageutils

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

## Load Images into a Matrix

In [2]:
base_dir = 'square_images128_traintest'
#(X_all, y_all, classes, class_dict) = imageutils.load_data(base_dir)
(X_train, y_train, classes, class_dict) = imageutils.load_data(base_dir + '/train')
(X_test, y_test, _, _) = imageutils.load_data(base_dir + '/test')
image_height = X_train.shape[1]
image_width = X_train.shape[2]
print(X_train.shape)
print(classes)

(1020, 128, 128, 3)
['bluebell', 'buttercup', 'colts_foot', 'cowslip', 'crocus', 'daffodil', 'daisy', 'dandelion', 'fritillary', 'iris', 'lily_valley', 'pansy', 'snowdrop', 'sunflower', 'tigerlily', 'tulip', 'windflower']


In [3]:
def extract_all_gist_features(X):
    num_examples = X.shape[0]
    features = []
    for index in range(num_examples):
        feature_vec = gist.extract(X[index].astype('uint8'))
        features.append(feature_vec)
    
    X_features = np.vstack(features)
    return X_features

In [4]:
def extract_gist(I):
    return gist.extract(I.astype('uint8'))

In [5]:
def tiny_image(image_array, width=16, height=16):
    I = Image.fromarray(image_array.astype('uint8'))
    I_tiny = I.resize((width, height))
    I_tiny_array = np.array(I_tiny).astype('float')
    return np.reshape(I_tiny_array, width * height * 3)

In [6]:
def color_histogram_rgb(image_array):
    h, edges = np.histogramdd(image_array.reshape(-1,3), 8, normed=True, range=[(0,255), (0,255), (0,255)])
    return h.flatten()

In [7]:
color_histogram_rgb(X_train[0]).shape

(512,)

In [8]:
def extract_features(X, feature_functions):
    num_examples = X.shape[0]
    feature_rows = []
    for index in range(num_examples):
        I = X[index]
        features = []
        for feature_func in feature_functions:
            feature_vec = feature_func(I)
            features.append(feature_vec)
        
        example_features_row_vec = np.hstack(features)
        feature_rows.append(example_features_row_vec)
    
    X_features = np.vstack(feature_rows)
    return X_features

In [9]:
from features import hog_feature, color_histogram_hsv

num_color_bins = 100 # Number of bins in the color histogram
#feature_fns = [extract_gist, hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
#feature_fns = [extract_gist, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
#feature_fns = [extract_gist]
#feature_fns = [lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
#feature_fns = [extract_gist, tiny_image]
feature_fns = [extract_gist, tiny_image, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_features = extract_features(X_train, feature_fns)
#X_val_feats = extract_features(X_val, feature_fns)
X_test_features = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_features = np.mean(X_train_features, axis=0)
mean_features = np.expand_dims(mean_features, axis=0)
X_train_features -= mean_features
#X_val_feats -= mean_feat
X_test_features -= mean_features

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_features = np.std(X_train_features, axis=0)
std_features = np.expand_dims(std_features, axis=0)
X_train_features /= std_features
#X_val_feats /= std_feat
X_test_features /= std_features

# Preprocessing: Add a bias dimension
X_train_features = np.hstack([X_train_features, np.ones((X_train_features.shape[0], 1))])
#X_val_feats = np.hstack([X_val_feats, np.ones((1, X_val_feats.shape[1]))])
X_test_features = np.hstack([X_test_features, np.ones((X_test_features.shape[0], 1))])


#X_train_features = X_train_features.T
#X_test_features = X_test_features.T

print(X_train_features.shape)

(1020, 1829)


In [20]:
from sklearn.svm import SVC
#from sklearn.cross_validation import cross_val_score

svm = SVC(kernel='linear', class_weight='balanced')
svm.fit(X_train_features, y_train)

y_pred = svm.predict(X_test_features)
print('accuracy: %f' % (np.mean(y_pred == y_test)))

accuracy: 0.708824


In [15]:
from sklearn.svm import SVC
#from sklearn.cross_validation import cross_val_score

svm = SVC(kernel='linear', class_weight='balanced', probability=True)
svm.fit(X_train_features, y_train)

y_pred = svm.predict_proba(X_test_features)
print(y_pred[0])
#print('accuracy: %f' % (np.mean(y_pred == y_test)))

[ 0.68050956  0.00236754  0.00752898  0.01382683  0.0071524   0.00244445
  0.02441084  0.00217947  0.04845797  0.06338281  0.03288691  0.03804714
  0.03154834  0.00359448  0.00398465  0.01450487  0.02317276]


In [16]:
y_augmented = [[(prob, index) for (index,prob) in enumerate(y_pred[index])] for index in range(len(y_pred))]
y_sorted = [sorted(y_augmented[index], reverse=True) for index in range(len(y_augmented))]
y_top5 = [y_sorted[index][:5] for index in range(len(y_sorted))]
y_top5_labels = [[label for (prob, label) in y_top5[index]] for index in range(len(y_sorted))]
print(y_top5_labels[0])

num_within_top_5 = 0.0
for index in range(len(y_test)):
    if y_test[index] in y_top5_labels[index]:
        num_within_top_5 += 1.0

percentage_within_top_5 = num_within_top_5 / len(y_test)
print(percentage_within_top_5)

[0, 9, 8, 11, 10]
0.947058823529


In [ ]:
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier(average=40, n_iter=100)
classifier.fit(X_train_features, y_train)

y_pred = classifier.predict(X_test_features)
print('accuracy: %f' % (np.mean(y_pred == y_test)))

In [21]:
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score

num_examples = X_train_features.shape[0]
print(num_examples)
cv = KFold(num_examples, n_folds=10, shuffle=True, random_state=None)

svm_cv_accuracies = cross_val_score(svm, X_train_features, y_train, cv=cv, n_jobs=7)
print(svm_cv_accuracies)

1020
[ 0.66666667  0.58823529  0.65686275  0.65686275  0.68627451  0.6372549
  0.65686275  0.55882353  0.6372549   0.59803922]


In [ ]:
from sklearn.grid_search import GridSearchCV

param_grid = {
    'C': [1e3, 5e3, 1e4, 5e4, 1e5],
    'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
}

#clf = GridSearchCV(svm, param_grid, scoring='f1', cv=cv, n_jobs=7)
clf = GridSearchCV(svm, param_grid, cv=cv, n_jobs=7)

clf = clf.fit(X_train_features, y_train)

print("Best estimator found by randomized hyper parameter search:")
print(clf.best_params_)
print("Best parameters validation score: {:.3f}".format(clf.best_score_))